**Mount to Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/MonashBuildings/

**Initialise the dataset**

In [ ]:
import torchvision.transforms as transforms
# ImageFolder treats images in different folder as different classes
from torchvision.datasets import ImageFolder

In [ ]:
# Define the transform to normalize the data
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the training data
trainset = ImageFolder(root = './train', transform = transform)

**Data Normalization**

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
batch_size = 4
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size,
                                          shuffle = True, num_workers = 2)

**Visualize the input**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
dataiter = iter(trainloader)
images,labels = next(dataiter)
# This is batch[1]

In [ ]:
print(images.shape)
print(labels.shape)
# There are 4 images in one batch

In [ ]:
dataiter = iter(trainloader)
images,labels = next(dataiter)
# This is batch[1]

def imshow(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
    img = img.numpy().transpose((1, 2, 0))  # Convert to numpy and reorder dimensions
    print(img.shape)
    img = std * img + mean  # Unnormalize
    img = np.clip(img, 0, 1)  # Clip values to be between 0 and 1
    plt.imshow(img)
    plt.show()

# Usage
imshow(images[0])
print(f'The label for is {labels[0]}')
imshow(images[2])
print(f'The label for is {labels[2]}')

**Set the CNN model**

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class CNN(nn.Module):
  def __init__(self, num_classes = 3):
    super().__init__()
    # First convolutional layer
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool_1 = nn.MaxPool2d(2, 2)

    # Second convolutional layer
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.pool_2 = nn.MaxPool2d(2, 2)

    # Fully connected layer
    self.fc1 = nn.Linear(53 * 53 * 16, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 3)

  def forward(self, x):
    # Pass the first convolutional layer
    x = self.pool_1(F.relu(self.conv1(x)))

    # Pass the second convolutional layer
    x = self.pool_2(F.relu(self.conv2(x)))

    # Flatten x into one dimension
    x = torch.flatten(x, 1)

    # Pass the fully connected layer
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)

    return x

  def predict(self, x):
    predictions = self(x)

    # Get the predicted classes
    _, predicted_classes = torch.max(predictions, dim=1)

    return predicted_classes, x


CNN_model =  CNN()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(CNN_model.parameters(), lr = 0.001, momentum =0.9)

**Begin training**

In [ ]:
epoch_loss_mat = []
for epoch in range(15):
  batch_loss = 0.0
  epoch_loss = 0.0

  # loop over all batches of the trainloader
  for i, data in enumerate(trainloader, 0):
    inputs, labels = data
    optimizer.zero_grad()

    # Forward pass
    outputs = CNN_model(inputs)
    loss = criterion(outputs, labels)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    batch_loss = loss.item()
    epoch_loss += loss.item()

    print(f'epoch: {epoch}, batch_{i}, batch_loss: {batch_loss:.3f}')
    batch_loss  = 0.0

  print(f'epoch loss:{epoch_loss} for epoch_{epoch}')
  epoch_loss_mat.append(epoch_loss)
  epoch_loss = 0.0

print("Finished Training")

In [ ]:
plt.plot(epoch_loss_mat)
plt.title('CNN model loss of each epoch')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

**Save the model**

In [ ]:
PATH = './CNN_model.pth'
torch.save(CNN_model.state_dict(), PATH)

In [ ]:
labels, images = CNN_model.predict(images)

In [ ]:
for i in range(len(labels)):
  imshow(images[i])
  print(f'label is {labels[i]}')